In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.regularizers as regularizers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

2.3.0


In [2]:
class_names = os.listdir('../TrainingSet/gdsc-aiseries/train')
print(class_names)
number_of_classes = len(class_names)

datagen = ImageDataGenerator(
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.3,
    horizontal_flip=True,
    rescale=1.0/255,
)

train_generator = datagen.flow_from_directory(
    '../TrainingSet/gdsc-aiseries/train',
    target_size=(32,32),
    batch_size=200,
    subset='training',
    class_mode='sparse'
)

validation_generator = datagen.flow_from_directory(
    '../TrainingSet/gdsc-aiseries/train',
    target_size=(32,32),
    batch_size=200,
    subset='validation',
    class_mode='sparse'
)

tf.config.list_physical_devices('GPU')

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Found 40000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
model = keras.models.Sequential([
    keras.layers.Input((32,32,3)),
    keras.layers.Conv2D(64, (5,5), padding='same',
                        kernel_regularizer=regularizers.l2(1e-3),
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (5,5), padding='same',
                        kernel_regularizer=regularizers.l2(1e-3),
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(128, (3,3), padding='same',
                        kernel_regularizer=regularizers.l2(1e-3),
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3,3), padding='same',
                        kernel_regularizer=regularizers.l2(1e-3),
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(256, (3,3), padding='same',
                        kernel_regularizer=regularizers.l2(1e-3),
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(256, (3,3), padding='same',
                        kernel_regularizer=regularizers.l2(1e-3),
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(256, (3,3), padding='same',
                        kernel_regularizer=regularizers.l2(1e-3),
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='elu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='elu'),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(number_of_classes, activation='softmax')
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        4864      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        102464    
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       7

In [5]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                                              loss='sparse_categorical_crossentropy',
                                              metrics=[keras.losses.SparseCategoricalCrossentropy(), 'accuracy'])

In [6]:
class stopCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}) :
        if (logs.get('val_accuracy') >= 0.99 and logs.get('accuracy') >= 0.99) :
            print('\nReached 99% accuracy so stopping training')
            self.model.stop_training = True

In [7]:
callback = stopCallback()

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='../Model/aiseries/version3.hdf5',
                                                               monitor='val_accuracy',
                                                               mode='max',
                                                               save_best_only=True)

history = model.fit(train_generator, 
                    epochs=1000, 
                    callbacks=[callback, model_checkpoint_callback], 
                    validation_data=validation_generator)

Epoch 1/1000
 68/200 [=========>....................] - ETA: 2:52 - loss: 4.6151 - sparse_categorical_crossentropy: 3.6490 - accuracy: 0.1607

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['loss'], color='orange', label='Train loss')
plt.plot(history.history['val_loss'], color='blue', label='Validation loss')
plt.legend()
plt.show()